# Exercise 1: Mining Frequent Patterns


We start with a small example in the form of a transactional data set.

In [ ]:
# Import the required libraries
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules

In [ ]:
# A very small data set in the form of a list (transactions) of sets (items)
dataset = [
    {"Beer", "Nuts", "Diapers"},
    {"Beer", "Coffee", "Diapers"},
    {"Beer", "Diapers", "Eggs"},
    {"Nuts", "Eggs", "Milk"},
    {"Nuts", "Coffee", "Diapers", "Eggs"},
]
dataset

From this data set, we want to extract the frequent itemsets. 

We use the library `mlxtend`, which contains an implementation for the two methods Apriori and FP-growth known from the lecture. In doing so, it is necessary to prepare the data set for the use of `apriori` or `fpgrowth`.

<div class="alert alert-block alert-info">
    
**Task 1:** 
    
Take a look at the `mlxtend` [documentation](http://rasbt.github.io/mlxtend/USER_GUIDE_INDEX/) for information on how dataset must be structured for `apriori`/`fpgrowth` and preprocess our `dataset` accordingly.
    
</div>


In [ ]:
# Preprocess the dataset
# Create a TransactionEncoder
transaction_encoder = TransactionEncoder()

# Use the TransactionEncoder to transform the dataset into a one-hot encoded NumPy boolean array
one_hot_encoded_dataset = transaction_encoder.fit(dataset).transform(dataset)

# Transform the one-hot encoded array into a pandas DataFrame
preprocessed_dataset = pd.DataFrame(
    one_hot_encoded_dataset, columns=transaction_encoder.columns_
)
preprocessed_dataset

After this preparation, the determination of the frequent itemset in our dataset is possible by using `apriori` or `fpgrowth`.

<div class="alert alert-block alert-info">

**Task 2:**
    
Using `apriori` or `fpgrowth` from `mlxtend`, determine the frequent itemsets in our dataset.
Use a `min_support` of 0.4.

</div>

In [ ]:
# Use apriori from mlxtend to determine the frequent itemsets in our dataset
frequent_itemsets = apriori(preprocessed_dataset, min_support=0.4, use_colnames=True)
frequent_itemsets

In [ ]:
# Use fpgrowth from mlxtend to determine the frequent itemsets in our dataset
frequent_itemsets = fpgrowth(preprocessed_dataset, min_support=0.4, use_colnames=True)
frequent_itemsets

In order to obtain frequent patterns from the frequent itemsets, the `mlxtend` function `association_rules` can be used.

<div class="alert alert-block alert-info">

**Task 3:**
    
Generate all association rules with a support of at least 0.4 from the frequent itemsets using `association_rules` from `mlxtend`.

</div>

In [ ]:
# Generate the association rules/frequent patterns
frequent_patterns = association_rules(
    frequent_itemsets, metric="support", min_threshold=0.4
)
frequent_patterns

What was quite easy in the small example is usually a bit more difficult in reality. For example, data is often not available in transactional form and must first be adapted accordingly.

Imagine the following scenario:

*You are an employee in the fictitious company Adventure Works GmbH.*
*Your job is to find out which of the company's products are frequently bought together.*
*To start with, the management wants you to find the ten most "relevant" product pairs bought together.*

*You get access to the OLTP database of the company.*
*The information about individual transactions can be found in the relation `TransactionHistory`.*
*The translation of ProductIDs into real product names can be done with the help of the relation `Product`.*

*Given are some helpful libraries and the code to connect to the OLTP database:*

In [ ]:
# Import helpful libraries
import os
import tempfile
import sqlite3
import urllib.request
import pandas as pd

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
# Create a temporary directory
dataset_folder = tempfile.mkdtemp()

# Build path to database
database_path = os.path.join(dataset_folder, "adventure-works.db")

# Get the database
urllib.request.urlretrieve(
    "https://github.com/FAU-CS6/KDD-Databases/raw/main/AdventureWorks/adventure-works.db",
    database_path,
)

# Open connection to the adventure-works.db
connection = sqlite3.connect(database_path)

<div class="alert alert-block alert-info">

**Task 4:**
    
Find the ten most "relevant" frequent patterns in the OLTP database of the fictitious Adventure Works GmbH.
You have to implement every step from loading the DataFrames to determining the Association Rules from the Frequent Itemsets.

</div>

First the records of `TransactionHistory` and `Product` have to be loaded.
Since we don't know anything about the relations we load all attributes and tuples for both datasets.

In [ ]:
# Load TransactionHistory into a DataFrame and display the first ten rows
transaction_history_df = pd.read_sql_query(
    "SELECT * FROM TransactionHistory", connection
)
transaction_history_df.head(10)

In [ ]:
# Load Product into a DataFrame and display the first ten rows
product_df = pd.read_sql_query("SELECT * FROM Product", connection)
product_df.head(10)

The `TransactionHistory` seems to contain information about individual transactions.
But it is not yet possible to see how it is possible to determine which products (probably identified via the `ProductID`) are purchased together.

The Product table can be used to map from `ProductID` to `Name`.

We might therefore assume that we are looking for products that are purchased in the same transaction.
The attribute `TransactionID` seems to uniquely identify each transaction.
We test this hypothesis by determining whether there are `TransactionID`s with more than one linked `ProductID`.

In [ ]:
# Check the TransactionHistory DataFrame for cases of several different ProductIDs for the same TransactionID
# First group the dataframe by TransactionID and aggregate the other columns by counting different values
transaction_history_df_grouped = transaction_history_df.groupby(
    ["TransactionID"]
).count()

# Then check if there are results there cells inb ProductID there the count of different values is greater than one
transaction_history_df_grouped[transaction_history_df_grouped["ProductID"] > 1]

# No results => There are no cases of several different ProductIDs for the same TransactionID

We find no cases of multiple different `ProductID`s for the same `TransactionID` so our first hypothesis not seem to be correct.
Apparently the `TransactionID` is the primary key for the `TransactionHistory` relation and one `TransactionID` cannot refer to different `ProductID`s.

When looking at `TransactionHistory` a second attribute stands out.
The `ReferenceOrderID` could identify the individual order and products that are part of the same order are obviously purchased together.

So let's test this new hypothesis as well.

In [ ]:
# Check the TransactionHistory DataFrame for cases of several different ProductIDs for the same ReferenceOrderID
# Now group the dataframe by ReferenceOrderID and aggregate the other columns by counting different values
transaction_history_df_grouped = transaction_history_df.groupby(
    ["ReferenceOrderID"]
).count()

# Then check again if there are results there cells inb ProductID there the count of different values is greater than one
transaction_history_df_grouped[transaction_history_df_grouped["ProductID"] > 1]

# 23249 results => There are multiple cases of different ProductIDs for the same ReferenceOrderID

Our new hypothesis seems to be correct.
In the next step we want to search for `ProductID`s that regularly occur in the same `ReferenceOrderID`, the frequent itemsets of our problem.

To determine our frequent itemsets using mlxtend we first need to do some preprocessing on `TransactionHistory`.

In [ ]:
# Aggregate the TransactionHistory to have a list of ProductIDs per ReferenceOrderID
products_per_order_df = (
    transaction_history_df.groupby("ReferenceOrderID")["ProductID"]
    .apply(list)
    .reset_index(name="ProductIDs")
    .set_index("ReferenceOrderID")
)
products_per_order_df

In [ ]:
# Apply one hot encoding to the prepared dataset by using the TransactionEncoder
# Create a TransactionEncoder
transaction_encoder = TransactionEncoder()

# Use the TransactionEncoder to transform the dataset into a one-hot encoded NumPy boolean array
one_hot_encoded_dataset = transaction_encoder.fit(
    products_per_order_df["ProductIDs"].tolist()
).transform(products_per_order_df["ProductIDs"].tolist())

# Transform the one-hot encoded array into a pandas DataFrame
preprocessed_dataset = pd.DataFrame(
    one_hot_encoded_dataset,
    columns=transaction_encoder.columns_,
    index=products_per_order_df.index,
)
preprocessed_dataset

After the preprocessing steps the frequent itemsets can now theoretically be determined.
But we definitely do not know which min_support to choose. 

Even by trial and error it is difficult to find a meaningful threshold here and we have only been told by our bosses to find the ten most "relevant" frequent patterns.

First of all we should determine rather too many itemsets than too few because it is easier to discard frequent itemsets later than to create additional ones.

In [ ]:
# Determine the frequent itemsets
frequent_itemsets = fpgrowth(preprocessed_dataset, min_support=0.01, use_colnames=True)
frequent_itemsets

Before using the frequent itemsets to determine frequent patterns we have to determine how to define "relevance" in the context of frequent patterns.

If our bosses wanted to know which ten patterns occur most frequently in our dataset the *support* would be the appropriate measure.
Did they wanted to know how certain one can be that Product A will end up in the shopping cart if Product B is already there?
Then the calculation of *confidence* would be more appropriate.
In addition there is a large number of other interestingness measures.

All in all, this question cannot be answered conclusively.
In practice a dialog with the management would be appropriate in order to narrow down more precisely what is meant by the most "relevant" ten patterns.

This ambiguity was intentionally used in the assignment to show that the assignment will often contain inaccuracies in the real world. 

While in the real world dialogue is the best solution we have no opportunity to consult with our fictitious bosses.
Therefore we do what is best for us and choose the simplest measure to apply: the support.

It is not important that we generate only the 10 rules with the highest support.
As long as they are part of your list, everything is fine.

In [ ]:
# Generate the association rules/frequent patterns
frequent_patterns = association_rules(
    frequent_itemsets, metric="support", min_threshold=0.02
)
frequent_patterns

Of course it is no problem at all to sort out extra patterns afterwards. 

In [ ]:
# Delete the extra patterns
frequent_patterns = frequent_patterns.nlargest(10, "support")
frequent_patterns

After completing the core task we tidy up the results, because management can not use the Information that `ProductID` 871 is often purchased in combination with `ProductID` 870 because these are internal database ids. 

To complete our task satisfactorily for the management we still need to enrich the `ProductID`s with their actual names.

In [ ]:
# Merge the ProductName into the frequent pattern df
# We have to transform the ids within the two columns to strings first
# (as we know that there is only one item per set this is pretty simple)
frequent_patterns["antecedents"] = frequent_patterns["antecedents"].apply(
    lambda x: list(x)[0]
)
frequent_patterns["consequents"] = frequent_patterns["consequents"].apply(
    lambda x: list(x)[0]
)

# After that we have to merge frequent_patterns with the product df
frequent_patterns = pd.merge(
    frequent_patterns, product_df, left_on="antecedents", right_on="ProductID"
)[
    [
        "antecedents",
        "Name",
        "consequents",
        "antecedent support",
        "consequent support",
        "support",
        "confidence",
        "lift",
        "leverage",
        "conviction",
    ]
]
frequent_patterns = frequent_patterns.rename(columns={"Name": "antecedents name"})
frequent_patterns = pd.merge(
    frequent_patterns, product_df, left_on="consequents", right_on="ProductID"
)[
    [
        "antecedents",
        "antecedents name",
        "consequents",
        "Name",
        "antecedent support",
        "consequent support",
        "support",
        "confidence",
        "lift",
        "leverage",
        "conviction",
    ]
]
frequent_patterns = frequent_patterns.rename(columns={"Name": "consequents name"})

# Print the df
frequent_patterns

Now the assignment is fully completed!
In the virtual scenario introduced in the beginning of this part you would now be able to report to the management that the `Mountain Bottle Cage` is often purchased in combination with the `Water Bottle - 30 oz.`.
The same is true for the other nine requested frequent patterns.



The library `mlxtend` offers some more measures besides support and confidence for the determination of frequent patterns.
While lift, leverage and conviction are offered, the kulczynski metric and imbalance ratio presented in the lecture are not.

We can use the antecedent support, the consequent support and the support calculated by `mlxtend` to easily calculate these interestingness measures.

<div class="alert alert-block alert-info">

**Task 5:** 
    
Write a function to compute the kulczynski measure known from the lecture as `Kulc(a, b)`.

</div>

In [ ]:
# Complete the function kulczynski_measure to compute the kulczynski measure
def kulczynski_measure(antecedent_support, consequent_support, support):
    # Simply use the formula introduced in the lecture
    return (support / 2) * ((1 / antecedent_support) + (1 / consequent_support))


# Compute the kulczynski measure for "Water Bottle - 30 oz." -> "Mountain Bottle Cage"
kulczynski_measure(0.112802, 0.049356, 0.041139)

<div class="alert alert-block alert-info">

**Task 6:**
    
Write a function to compute the imbalance ratio.

</div>

In [ ]:
# Complete the function imbalance_ratio to compute the imbalance ratio
def imbalance_ratio(antecedent_support, consequent_support, support):
    # Simply use the formula introduced in the lecture
    return abs(antecedent_support - consequent_support) / (
        antecedent_support + consequent_support - support
    )


# Compute the imbalance ratio for "Water Bottle - 30 oz." -> "Mountain Bottle Cage"
imbalance_ratio(0.112802, 0.049356, 0.041139)

Now the question arises as to how these metrics have to be interpreted.

<div class="alert alert-block alert-info">

**Task 7:**
    
Interpret the interestingness measures for the association rule `"Water Bottle - 30 oz."` $\rightarrow$ `"Mountain Bottle Cage"`

</div>

First both values must be interpreted separately from each other:

- **Kulczynski Measure:**  
When kulczynski measure is close to 0 or 1 we have an "interesting" association rule.
Since in this case the value is about 0.6, the kulczynski measure rather suggests that this association rule is uninteresting. 
- **Imbalance Ratio:**  
For the imbalance ratio, a value of 0 indicates a perfectly balanced association rule while 1 indicates a very unbalanced one.
In this case we are about 0.52, which is about the middle of the spectrum.
Thus we cannot speak of a particularly well balanced rule, but neither can we speak of a completely unbalanced one.

In summary we have not discovered the most interesting rule, but one that is not completely uninteresting (this would the case for kulczynski measure = 0.5 and imbalance ratio = 0.0). 